Este notebook contiene la solución completa del problema de clasificación, desde la carga y preprocesamiento de los datos hasta la selección automática del mejor modelo y la generación del archivo submission.csv para enviar a Kaggle, se realiza una validación cruzada para comparar modelos y se selecciona el de mejor rendimiento y para evitar problemas de memoria se utiliza una muestra representativa de 80.000 registros del conjunto de entrenamiento.

In [ ]:
#Carga datos desde Google Drive

from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, cross_validate, ShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np


train_path = '/content/drive/MyDrive/Proyecto IA/Pruebas_Saber_Pro/train.csv'
test_path = '/content/drive/MyDrive/Proyecto IA/Pruebas_Saber_Pro/test.csv'


df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)
print("Train:", df_train.shape, "Test:", df_test.shape)

# Usa una muestra para evitar exceso de memoria
df_train = df_train.sample(80000, random_state=42)

#Preprocesamiento básico y limpieza

# Eliminar filas sin etiqueta
df_train = df_train.dropna(subset=['RENDIMIENTO_GLOBAL'])

# Imputa valores nulos
for df in [df_train, df_test]:
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].fillna('Desconocido')
        else:
            df[col] = df[col].fillna(df[col].median())

# Guardar ID del test para el archivo de predicción
test_ids = df_test['ID']

#Codificación One-Hot y normalización

# Unimos train y test para procesarlos juntos
df_train['is_train'] = 1
df_test['is_train'] = 0
df_test['RENDIMIENTO_GLOBAL'] = 'desconocido'

df_all = pd.concat([df_train, df_test])

# Identifica columnas categóricas
cat_cols = df_all.select_dtypes(include='object').columns.tolist()
cat_cols.remove('RENDIMIENTO_GLOBAL')
if 'ID' in cat_cols:
    cat_cols.remove('ID')

# One-hot encoding con reducción de columnas redundantes
df_all = pd.get_dummies(df_all, columns=cat_cols, drop_first=True)

# Normaliza columnas numéricas
num_cols = df_all.select_dtypes(include=['int64', 'float64']).columns.tolist()
if 'ID' in num_cols:
    num_cols.remove('ID')

scaler = MinMaxScaler()
df_all[num_cols] = scaler.fit_transform(df_all[num_cols])

#Separa nuevamente los conjuntos

df_train = df_all[df_all['is_train'] == 1].drop(columns=['is_train'])
df_test = df_all[df_all['is_train'] == 0].drop(columns=['is_train', 'RENDIMIENTO_GLOBAL'])

X = df_train.drop(columns=['RENDIMIENTO_GLOBAL', 'ID'])
y = df_train['RENDIMIENTO_GLOBAL']
X_test = df_test.drop(columns=['ID'])

# División en entrenamiento/validación y prueba final

X_trainval, X_test_final, y_trainval, y_test_final = train_test_split(X, y, test_size=0.2, random_state=42)
val_size = 0.2 / 0.8

#Selección del mejor modelo

def report_cv_score(z):
    print("Test score %.3f (±%.4f) con %d splits" % (np.mean(z['test_score']), np.std(z['test_score']), len(z['test_score'])))
    print("Train score %.3f (±%.4f)" % (np.mean(z['train_score']), np.std(z['train_score'])))

# Usa una muestra para comparar modelos rápidamente
X_sample = X_trainval.sample(15000, random_state=42)
y_sample = y_trainval.loc[X_sample.index]

models = [
    ('Random Forest', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('Linear SVM', SVC(kernel='linear', probability=True, random_state=42)),
    ('Logistic Regression', LogisticRegression(max_iter=1000, random_state=42))
]

scores = []
for name, model in models:
    print(f"\nEvaluando: {name}")
    z = cross_validate(
        model,
        X_sample,
        y_sample,
        cv=ShuffleSplit(n_splits=3, test_size=val_size),
        scoring='accuracy',
        return_train_score=True
    )
    report_cv_score(z)
    scores.append(np.mean(z['test_score']))

#Selección automática del mejor modelo
best_index = np.argmax(scores)
best_model_name, best_model = models[best_index]
print(f"\n Modelo seleccionado: {best_model_name}")

#Evaluación final del modelo

best_model.fit(X_trainval, y_trainval)
y_pred_final = best_model.predict(X_test_final)

acc = accuracy_score(y_test_final, y_pred_final)
print(f" Desempeño del modelo seleccionado en test final: {acc:.4f}")

#8Entrenamiento final y generación del archivo

best_model.fit(X, y)
final_predictions = best_model.predict(X_test)

submission = pd.DataFrame({
    'ID': test_ids,
    'RENDIMIENTO_GLOBAL': final_predictions
})

submission.to_csv('submission.csv', index=False)










Mounted at /content/drive
Train: (692500, 21) Test: (296786, 20)

Evaluando: Random Forest
Test score 0.388 (±0.0037) con 3 splits
Train score 1.000 (±0.0000)

Evaluando: Linear SVM
Test score 0.399 (±0.0033) con 3 splits
Train score 0.486 (±0.0019)

Evaluando: Logistic Regression
Test score 0.416 (±0.0013) con 3 splits
Train score 0.483 (±0.0024)

 Modelo seleccionado: Logistic Regression
 Desempeño del modelo seleccionado en test final: 0.4219
✅ Archivo 'submission.csv' generado exitosamente.
